# [Merge and filter data](#merge-and-filter-data)

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from glob import glob
from IPython.display import display

import altair as alt
import pandas as pd

In [ ]:
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
alt.renderers.enable("notebook")

<a id="toc"></a>

## [Table of Contents](#table-of-contents)
0. [About](#about)
1. [User Inputs](#user-inputs)
2. [Merge and filter Hubble data](#merge-and-filter-hubble-data)
3. [Merge and filter NYTimes data](#merge-and-filter-nytimes-data)
4. [Merge and filter Guardian data](#merge-and-filter-guardian-data)
5. [Merge and filter Space.com data](#merge-and-filter-space.com-data)

<a id="about"></a>

## 0. [About](#about)

In this notebook, we will merge scraped listings data (including metadata), from various news publications and stored in `data/raw`, into a separate `data/processed/*_processed.csv` file per news publication and filter out any articles that is less than 500 words in length

<a id="user-inputs"></a>

## 1. [User Inputs](#user-inputs)

We'll define below the variables to be used throughout the code.

In [ ]:
# General inputs
PROJ_ROOT_DIR = os.getcwd()
data_dir = os.path.join(PROJ_ROOT_DIR, "data", "raw")
processed_data_dir = os.path.join(PROJ_ROOT_DIR, "data", "processed")

# Hubble Filenames
hubble_filename = "hubble_urls.csv"
hubble_text = "hubble.csv"
hubble_processed_filename = "hubble_processed.csv"

# NY Times Filenames
nytimes_filename = "nytimes_urls__*.csv"
nytimes_text_filenames = [
    "nytimes.csv",
    # # use below if you scrape only certain articles' text at once
    # # and then want to combine all tries together
    # "nytimes_1.csv",
    # "nytimes_3.csv",
    # "nytimes_2.csv"
]
nytimes_processed_filename = "nytimes_processed.csv"

# Space.com Filenames
space_filename = "space_com_urls.csv"
space_text_filenames = [
    "space.csv",
    # # use below if you scrape only certain articles' text at once
    # # and then want to combine all tries together
    # "space_1.csv",
    # "space_2.csv",
    # "space_3.csv",
    # "space_4.csv",
    # "space_5.csv",
]
space_processed_filename = "space_processed.csv"

# Guardian Filenames
guardian_filename = "guardian_urls.csv"
guardian_text_filenames = [
    "guardian.csv"
    # # use below if you scrape only certain articles' text at once
    # # and then want to combine all tries together
    # "guardian_1.csv",
    # "guardian_2.csv"
]
guardian_processed_filename = "guardian_processed.csv"

<a id="merge-and-filter-hubble-data"></a>

## 2. [Merge and filter Hubble data](#merge-and-filter-hubble-data)

We'll start by loading the scraped text and listings urls from the Hubble website into separate `DataFrame`s.

In [ ]:
df_hubble_listings = pd.read_csv(os.path.join(data_dir, hubble_filename)
df_hubble_text = pd.read_csv(os.path.join(processed_data_dir, hubble_text)

We'll then rename columns

In [ ]:
df_hubble_listings.rename(
    columns={"publication": "publication_date", "mission": "publication"}, inplace=True
)
df_hubble_text.drop(["publication_date"], axis=1, inplace=True)
display(df_hubble_listings.head(2))
display(df_hubble_text.head(2))

Next, we'll set an index for the `DataFrame`s so that we can join each `DataFrame` on its index

In [ ]:
df_hubble_text = df_hubble_text.set_index(["url", "publication"])
df_hubble_listings = df_hubble_listings.set_index(["url", "publication"])
print(df_hubble_text.shape)
display(df_hubble_text.head(2))
print(df_hubble_listings.shape)
display(df_hubble_listings.head(2))

Next, we'll merge the `DataFrame`s on the index and reset the index so that these index columns appear in the merged `DataFrame`

In [ ]:
df_hubble = df_hubble_text.merge(
    df_hubble_listings, left_index=True, right_index=True, how="inner",
).reset_index(drop=False)
print(df_hubble.shape)
display(df_hubble.head(2))

Next, we'll append `datetime` attributes as columns to the merged `DataFrame`

In [ ]:
L = ["year", "month", "day", "dayofweek", "dayofyear", "weekofyear", "quarter"]
df_hubble.drop(L, axis=1, inplace=True)
df_hubble["publication_date"] = pd.to_datetime(df_hubble["publication_date"], utc=True)

In [ ]:
df_hubble = df_hubble.join(
    pd.concat(
        (getattr(df_hubble["publication_date"].dt, i).rename(i) for i in L), axis=1
    )
)
df_hubble["decade"] = df_hubble["year"] // 10 * 10
print(df_hubble.shape)
df_hubble.head(3)

Next, we'll filter out news articles of less than 500 characters

In [ ]:
df_hubble = df_hubble[(df_hubble["text"].str.len() > 500)]

Next, we'll drop unwanted columns from the merged `DataFrame`

In [ ]:
unwanted_hubble_cols = [
    "url",
    "publication",
    "news_id",
    "publication_date",
    "day",
    "dayofweek",
    "dayofyear",
    "weekofyear",
    "quarter",
]
df_hubble.drop(unwanted_hubble_cols, axis=1, inplace=True)
print(df_hubble.shape)

Finally, we'll export the merged `DataFrame` to a `.csv` file

In [ ]:
print(
    f"Memory footprint of DataFrame: {(df_hubble.memory_usage().sum() / 1000 / 1000):.2f} MB"
)
df_hubble.to_csv(os.path.join(processed_data_dir, hubble_processed_filename), index=False)

In [ ]:
df_hubble_loaded = pd.read_csv(os.path.join(processed_data_dir, hubble_processed_filename))
print(
    f"Memory footprint of DataFrame: {(df_hubble_loaded.memory_usage().sum() / 1000 / 1000):.2f} MB"
)

<a id="merge-and-filter-nytimes-data"></a>

## 3. [Merge and filter NYTimes data](#merge-and-filter-nytimes-data)

We'll start by loading the scraped text and listings urls from the New York Times website into separate `DataFrame`s.

In [ ]:
df_nytimes_listings = pd.concat(
    [pd.read_csv(os.path.join(data_dir, f)) for f in glob(os.path.join(data_dir, nytimes_filename))]
)
df_nytimes_text = pd.concat([pd.read_csv(os.path.join(data_dir, f)) for f in nytimes_text_filenames])

We'll then rename columns

In [ ]:
df_nytimes_listings.rename(
    columns={"web_url": "url", "source": "publication"}, inplace=True
)
df_nytimes_text["publication"] = df_nytimes_text["publication"].str.replace(
    "nytimes", "The New York Times"
)
display(df_nytimes_listings.head(2))
display(df_nytimes_text.head(2))

Next, we'll set an index for the `DataFrame`s so that we can join each  `DataFrame` on its index

In [ ]:
df_nytimes_text = df_nytimes_text.set_index(["url", "publication"])
df_nytimes_listings = df_nytimes_listings.set_index(["url", "publication"])
print(df_nytimes_text.shape)
display(df_nytimes_text.head(2))
print(df_nytimes_listings.shape)
display(df_nytimes_listings.head(2))

Next, we'll merge the `DataFrame`s on the index and reset the index so that these index columns appear in the merged `DataFrame`

In [ ]:
df_nytimes = df_nytimes_text.merge(
    df_nytimes_listings, left_index=True, right_index=True, how="inner",
).reset_index(drop=False)
print(df_nytimes.shape)
display(df_nytimes.head(2))

Next, we'll append `datetime` attributes as columns to the merged `DataFrame`

In [ ]:
df_nytimes_text["decade"] = df_nytimes_text["year"] // 10 * 10

Next, we'll filter out news articles of less than 500 characters and exclude unwanted articles (i.e. we'll remove articles from the subsection `Environment`)

In [ ]:
df_nytimes = df_nytimes[
    (df_nytimes["type_of_material"] == "News")
    & (df_nytimes["subsection_name"] != "Environment")
    & (df_nytimes["text"].str.len() > 500)
]

Next, we'll drop unwanted columns from the merged `DataFrame`

In [ ]:
unwanted_nytimes_cols = [
    "url",
    "section_name",
    "page",
    "news_desk",
    "document_type",
    "type_of_material",
    "publication_date",
    "day",
    "dayofweek",
    "dayofyear",
    "weekofyear",
    "quarter",
    "lead_paragraph",
]
df_nytimes.drop(unwanted_nytimes_cols, axis=1, inplace=True)
print(df_nytimes.shape)
df_nytimes.head(2)

Finally, we'll export the merged `DataFrame` to a `.csv` file

In [ ]:
print(
    f"Memory footprint of DataFrame: {(df_nytimes.memory_usage().sum() / 1000 / 1000):.2f} MB"
)
df_nytimes.to_csv(os.path.join(processed_data_dir, nytimes_processed_filename), index=False)

In [ ]:
df_nytimes_loaded = pd.read_csv(os.path.join(processed_data_dir, nytimes_processed_filename))
print(
    f"Memory footprint of DataFrame: {(df_nytimes_loaded.memory_usage().sum() / 1000 / 1000):.2f} MB"
)

<a id="merge-and-filter-guardian-data"></a>

## 4. [Merge and filter Guardian data](#merge-and-filter-guardian-data)

We'll start by loading the scraped text and listings urls from the Guardian website into separate `DataFrame`s.

In [ ]:
df_guardian_listings = pd.read_csv(os.path.join(data_dir, guardian_filename))
df_guardian_text = pd.concat(
    [pd.read_csv(os.path.join(data_dir, f)) for f in guardian_text_filenames]
)

We'll then rename columns

In [ ]:
df_guardian_listings.rename(
    columns={"webUrl": "url", "webPublicationDate": "publication_date"}, inplace=True
)
df_guardian_text.drop(["publication_date"], axis=1, inplace=True)
display(df_guardian_listings.head(2))
display(df_guardian_text.head(2))

Next, we'll set an index for the `DataFrame`s so that we can join each `DataFrame` on its index

In [ ]:
df_guardian_text = df_guardian_text.set_index(["url"])
df_guardian_listings = df_guardian_listings.set_index(["url"])
print(df_guardian_text.shape)
display(df_guardian_text.head(2))
print(df_guardian_listings.shape)
display(df_guardian_listings.head(2))

Next, we'll merge the `DataFrame`s on the index and reset the index so that these index columns appear in the merged `DataFrame`

In [ ]:
df_guardian = df_guardian_text.merge(
    df_guardian_listings, left_index=True, right_index=True, how="inner",
).reset_index(drop=False)
print(df_guardian.shape)
display(df_guardian.head(2))

Next, we'll append `datetime` attributes as columns to the merged `DataFrame`

In [ ]:
L = ["year", "month", "day", "dayofweek", "dayofyear", "weekofyear", "quarter"]
df_guardian.drop(L, axis=1, inplace=True)
df_guardian["publication_date"] = pd.to_datetime(
    df_guardian["publication_date"], utc=True
)

In [ ]:
df_guardian = df_guardian.join(
    pd.concat(
        (getattr(df_guardian["publication_date"].dt, i).rename(i) for i in L), axis=1
    )
)
df_guardian["decade"] = df_guardian["year"] // 10 * 10
print(df_guardian.shape)
df_guardian.head(3)

Next, we'll filter out news articles of less than 500 characters

In [ ]:
df_guardian = df_guardian[df_guardian["text"].str.len() > 500]

Next, we'll drop unwanted columns from the merged `DataFrame`

In [ ]:
unwanted_guardian_cols = [
    "url",
    "id",
    "sectionId",
    "sectionName",
    "type",
    "isHosted",
    "pillarId",
    "pillarName",
    "page",
    "publication_date",
    "day",
    "dayofweek",
    "dayofyear",
    "weekofyear",
    "quarter",
]
df_guardian.drop(unwanted_guardian_cols, axis=1, inplace=True)
print(df_guardian.shape)

Finally, we'll export the merged `DataFrame` to a `.csv` file

In [ ]:
print(
    f"Memory footprint of DataFrame: {(df_guardian.memory_usage().sum() / 1000 / 1000):.2f} MB"
)
df_guardian.to_csv(os.path.join(processed_data_dir, guardian_processed_filename), index=False)

In [ ]:
df_guardian_loaded = pd.read_csv(os.path.join(processed_data_dir, guardian_processed_filename))
print(
    f"Memory footprint of DataFrame: {(df_guardian_loaded.memory_usage().sum() / 1000 / 1000):.2f} MB"
)

<a id="merge-and-filter-space.com-data"></a>

## 5. [Merge and filter Space.com data](#merge-and-filter-space.com-data)

We'll start by loading the scraped text and listings urls from the Space.com website into separate `DataFrame`s.

In [ ]:
df_space_listings = pd.read_csv(os.path.join(data_dir, space_filename))
df_space_text = pd.concat([pd.read_csv(os.path.join(data_dir, f)) for f in space_text_filenames])
# df_space_text.drop(["publication_date"], axis=1, inplace=True)
display(df_space_listings.head(2))
display(df_space_text.head(2))

Next, we'll set an index for the `DataFrame`s so that we can join each on the index

In [ ]:
df_space_text = df_space_text.set_index(["url"])
df_space_listings = df_space_listings.set_index(["url"])
print(df_space_text.shape)
display(df_space_text.head(2))
print(df_space_listings.shape)
display(df_space_listings.head(2))

Next, we'll merge the `DataFrame`s on the index and reset the index so that these index columns appear in the merged `DataFrame`

In [ ]:
df_space = df_space_text.merge(
    df_space_listings, left_index=True, right_index=True, how="inner",
).reset_index(drop=False)
print(df_space.shape)
display(df_space.head(2))

Next, we'll append `datetime` attributes as columns to the merged `DataFrame`

In [ ]:
df_space["decade"] = df_space["year"] // 10 * 10

Next, we'll filter out news articles of less than 500 characters

In [ ]:
df_space = df_space[df_space["text"].str.len() > 500]

Next, we'll drop unwanted columns from the merged `DataFrame`

In [ ]:
unwanted_space_cols = [
    "url",
    "publication_date",
    "publication",
    "archive_url",
    "day",
    "dayofweek",
    "dayofyear",
    "weekofyear",
    "quarter",
]
df_space.drop(unwanted_space_cols, axis=1, inplace=True)
print(df_space.shape)
df_space.head(2)

Finally, we'll export the merged `DataFrame` to a `.csv` file

In [ ]:
print(
    f"Memory footprint of DataFrame: {(df_space.memory_usage().sum() / 1000 / 1000):.2f} MB"
)
df_space.to_csv(os.path.join(processed_data_dir, space_processed_filename), index=False)

In [ ]:
df_space_loaded = pd.read_csv(os.path.join(processed_data_dir, space_processed_filename))
print(
    f"Memory footprint of DataFrame: {(df_space_loaded.memory_usage().sum() / 1000 / 1000):.2f} MB"
)